In [ ]:
import sys
!conda install --yes --prefix {sys.prefix} rasterio

!conda install rasterio -y # do not install from conda-forge, it has an old version of the package. This only works for python 3.5, not yet 3.6-> will install rasterio 0.36!
!conda install fiona -y # this iss for reading shapefiles (vector data)
!conda install rasterstats -y # this is for zonal operations on rasters  (used later)

In [4]:
!conda install matplotlib -y


Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.



PackagesNotFoundError: The following packages are not available from current channels:

  - matplot

Current channels:

  - https://repo.anaconda.com/pkgs/main/win-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/win-64
  - https://repo.anaconda.com/pkgs/r/noarch
  - https://repo.anaconda.com/pkgs/msys2/win-64
  - https://repo.anaconda.com/pkgs/msys2/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




## Working with raster data - matrices of values
here, we will work with the SRTM DEM data (heights)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import rasterio
from rasterio import plot

In [ ]:
import os
raster_path = os.path.join("/home/nbuser/library/L5","data_L5","EastofMelbSRTM.tif")

os.path.exists(raster_path)

In [ ]:
src = rasterio.open(raster_path,"r") #opened to read only

In [ ]:
%matplotlib inline
plot.show(src)


In [ ]:
# it is useful to find out how your data are structured
# number of bands (some datasets can have RGB bands and more)
src.count

In [ ]:
# plot a histogram of the values in the raster
plot.show_hist(src)

In [ ]:
help(plot.show_hist)

In [ ]:
# raster shape in crs above
src.bounds

In [ ]:
# let's visualize some part of it - a greyscale, and with some contour lines
fig, ax = plt.subplots(1, figsize=(12, 12))
plot.show((src, 1), cmap='Greys_r', interpolation='none', ax=ax)
ax.set_xlim(145.5,146)
ax.set_ylim(-37.0,-37.5)
plot.show((src, 1),contour=True, ax=ax)

In [ ]:
# name of dataset - alwasy hanbdy to access, if yo uare processing many
src.name

In [ ]:
# coordinate system
src.crs

In [ ]:
# value for missing data
src.nodata

In [ ]:
# size of dataset
src.shape

## Side note - Numpy arrays

In [ ]:
# rasters are based on NP arrays. Let's see
x = np.array([[1, 2, 3], [4, 5, 6]], np.int32)
# arrays are indexed from 0 as in all Python.

In [ ]:
x

In [ ]:
x[1,2] # get the second row, third column

In [ ]:
# now the same with our raster
src = rasterio.open(raster_path, 'r+')
arr = src.read() # gets the array read in memory - careful,  may be inefficient
print(arr.shape) # check the shape - see that it is one band, and the size of the array
pt_top_left = arr[0,0,0]
pt_top_left
# Beware, arrays are mutable - you can alter the values unless you are in read only mode!

In [ ]:
# if you open input outside of context, you must make sure that you close it.
src.close()

## Using contexts 
Contexts for reading/writing makes the structure more undertndable and makes sure you close the file

In [ ]:
# using a context syntax to read
# documentation for the modes: http://www.manpagez.com/man/3/fopen/ 
with rasterio.open(raster_path, 'r+') as r: # note the mode
    print(r.closed) # see whether you can read/read/write or write to this
    print(r.mode)
    arr = r.read()  # read all raster values
    print(arr.shape)  # this is a numpy array, with dimensions [band, row, col]
    idx_pt = src.index(145.5,-37.5) # index of the cell containing coordinates: 144.5,-35.5 NOTE: in rasterio 1 this is a function src.xy()
    print("Index of a cell at a given point - here near middle of the raster: "+str(idx_pt[0])+" "+str(idx_pt[1]) ) # 
    coords_pt = src.ul(idx_pt[0],idx_pt[1])  
    print("coordinates of the upper left cell where the index is are: ",coords_pt) # print max and min value of raster
    # alrternatively
    print(src.affine * idx_pt) # using athe affine transformation function

## Masking by vector feature from shapefile

In [ ]:
# compute the statistics by area - using rasterstats
from rasterstats import zonal_stats
zonal_stats("data_raster/melb_shape.shp", 'EastofMelbSRTM.tif',
            stats="count min mean max median")


In [ ]:
## this is a way to read shp data in - from example above. We are going to disucss this more later, for now, just have a look at it.
with fiona.open("data_raster/melb_shape.shp", "r") as shapefile:
    features = [feature["geometry"] for feature in shapefile]

In [ ]:
features

# convolution using scipy

In [ ]:
import scipy.signal as sig 

In [ ]:
with rasterio.open('EastofMelbSRTM.tif', 'r+') as src:
    data = src.read()[0]# your first array with data
    kernel = np.ones((3,3))/9
    C = sig.convolve2d(data,kernel, 'valid')

In [ ]:
plot.show(C)

In [ ]:
help(sig.convolve2d)

## Raindrops
find where rain flows.
taken from http://www2.geog.ucl.ac.uk/~plewis/geogg122-2011-12/dem2.html 

In [ ]:
def padding(dem,size=1):
    '''
    Apply a border of size [pixel] to a spatial dataset

    Return the padded data with the original centred in the array
    '''
    out = np.zeros([dem.shape[0]+2*size,dem.shape[1]+2*size])
    out[:,:] = np.max(dem)+1
    out[size:-size,size:-size] = dem
    return out

In [ ]:
def localMin(dem):
    '''
    We wish to return the location of the minimum grid value
    in a neighbourhood.

    We assume the array is 2D and defined (y,x)

    We return wx,wy which are the cell displacements in x and y directions.

    '''
    wy = np.zeros_like(dem).astype(int)
    wx = np.zeros_like(dem).astype(int)
    winx = np.ones([3,3])
    for i in range(3):
        winx[:,i] = i - 1
    winy = winx.transpose()
    demp = padding(dem,size=1)
    for y in np.arange(1,demp.shape[0]-1):
        for x in np.arange(1,demp.shape[1]-1):
            win = demp[y-1:y+2,x-1:x+2]
            ww = np.where(win == np.min(win))
            whereX = winx[ww][0]
            whereY = winy[ww][0]
            wy[y-1,x-1] = whereY
            wx[y-1,x-1] = whereX
    return wx,wy


In [ ]:
with rasterio.open('EastofMelbSRTM.tif', 'r+') as src:
    data = src.read()[0]# your first band array with data
    (wx,wy) = localMin(data)
    plot(wx)

# slope and aspect
    based on: http://www2.geog.ucl.ac.uk/~plewis/geogg122-2011-12/dem2.html

In [ ]:
def gaussianFilter(sizex,sizey=None,scale=0.333):
    '''
    Generate and return a 2D Gaussian function
    of dimensions (sizex,sizey)

    If sizey is not set, it defaults to sizex
    A scale can be defined to widen the function (default = 0.333)
    '''
    sizey = sizey or sizex
    x, y = np.mgrid[-sizex:sizex+1, -sizey:sizey+1]
    g = np.exp(-scale*(x**2/float(sizex)+y**2/float(sizey)))
    return g/g.sum()

In [ ]:
def grad2d(dem):
    '''
    Calculate the slope and gradient of a DEM
    '''
    from scipy import signal
    f0 = gaussianFilter(3)
    I = signal.convolve(dem,f0,mode='valid') # applies smooothing by gaussian filter
    f1 = np.array([[-1,0,1],[-2,0,2],[-1,0,1]]) # SOBEL FILTER 
    f2 = f1.transpose()
    g1 = signal.convolve(I,f1,mode='valid')
    g2 = signal.convolve(I,f2,mode='valid')
    slope = np.sqrt(g1**2 + g2**2)
    aspect = np.arctan2(g2,g1)
    return slope, aspect

In [ ]:
slope, aspect = grad2d(A)

In [ ]:
plot.show(slope)

In [ ]:
result = scipy.ndimage.convolve(your_raster_as_numpy_array, weights=kernel) / kernel.size

In [ ]:
from scipy.ndimage.filters import generic_filter as gf

kernel = np.ones((3,3))
circular_mean = gf(data, np.mean, footprint=kernel) # size=(3,3)